In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys, os
utils_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm/src/utils")
sys.path.append(str(utils_path))

In [3]:
import pandas as pd
import numpy as np

from glob import glob

from dotenv import load_dotenv
from openai import OpenAI
import json

from pdf2image import convert_from_path
from PIL import Image

from langchain_core.documents import Document
# from langchain.vectorstores import FAISS

from utils import generate_random_name
from call_gpt4o import call_openai_endpoint

## Define Paths

In [4]:
base_path = Path("C:/Users/shaur/Desktop/Learnings/KG_Tests/senior_living_dm")
artifacts_path = os.path.join(base_path, "artifacts")
poppler_path = os.path.join(artifacts_path, "libraries/Release-23.08.0-0/poppler-23.08.0/Library/bin")
supp_apps_pdfs_dir_path = os.path.join(artifacts_path, "applications/pdfs")
supp_apps_pngs_dir_path = os.path.join(artifacts_path, "applications/pngs")
supp_apps_chunked_pngs_dir_path = os.path.join(artifacts_path, "applications/chunked_pngs")

## PDF to PNG

In [5]:
example_pdf_path = os.path.join(supp_apps_pdfs_dir_path, "crc_app.pdf")

In [6]:
# example_pdf_pages = convert_from_path(pdf_path=example_pdf_path, poppler_path=poppler_path)
# example_pdf_name = os.path.splitext(os.path.basename(example_pdf_path))[0]
# c=1
# for page in example_pdf_pages:
#     os.makedirs(supp_apps_pngs_dir_path, exist_ok=True)
#     img_name = f"{example_pdf_name}_{c}.png"
#     page.save(os.path.join(supp_apps_pngs_dir_path, img_name), "PNG")
#     c+=1

## Split PNGs

In [6]:
def split_img(n:int, alpha:int) -> None:

    for page_img_name in os.listdir(supp_apps_pngs_dir_path):
        page_img_path = os.path.join(supp_apps_pngs_dir_path, page_img_name)
        page_img = Image.open(page_img_path)
        w, h = page_img.size

        overlap = h * alpha // 100
        h = h//n
        

        for i in range(1, n+1):
            chunked_img_name = page_img_name.replace(".png", f"_{i}.png")
            chunked_img_path = os.path.join(supp_apps_chunked_pngs_dir_path, chunked_img_name)
            if i==1:
                chunked_img = page_img.crop((0, 0, w, h+overlap))
            elif i==n:
                chunked_img = page_img.crop((0, h*(i-1)-overlap, w, h*i))
            else:
                chunked_img = page_img.crop((0, h*(i-1)-overlap, w, h*i+overlap))
            os.makedirs(supp_apps_chunked_pngs_dir_path, exist_ok=True)
            chunked_img.save(chunked_img_path)

In [7]:
# split_img(n=5, alpha=5)

## Test Open AI

In [8]:
example_png_path = os.path.join(supp_apps_chunked_pngs_dir_path, "crc_app_2_2.png")

ans = call_openai_endpoint(example_png_path)

In [9]:
print(ans)

{
  "sub_acute_short_term_rehab": {
    "total_licensed_beds": 23,
    "average_annual_occupancy": 10
  },
  "skilled_nursing_intermediate_care": {
    "total_licensed_beds": 15,
    "average_annual_occupancy": 20
  },
  "assisted_living": {
    "total_licensed_beds": 32,
    "average_annual_occupancy": 12
  },
  "independent_care": {
    "total_units": 5,
    "total_residents_at_full_occupancy": 65,
    "common_dining_facilities": true,
    "individual_units_have_cooking_appliances": true
  }
}
